# Social Network Analysis for Data Warehouses

## Network Connectivity Analysis: Betweenness Centrality

In [ ]:
# Network Influencers: betweenness centrality

bet_cent = nx.betweenness_centrality(graph, normalized=True, endpoints=True)

# Print the labels of the nodes with the highest betweenness centrality.
sorted(bet_cent, key=bet_cent.get, reverse=True)[:5]

In [ ]:
def draw_betweenness_centrality_based_network(graph: Graph) -> None:
    node_color = [20000 * degree for _, degree in graph.out_degree]

    bet_cent = nx.betweenness_centrality(graph, normalized=True, endpoints=True)
    node_size = [node_bc * 1000000 for node_bc in bet_cent.values()]
    
    pos = nx.spring_layout(graph)
    plt.figure(figsize=(12, 8))
    nx.draw_networkx(graph, pos=pos, with_labels=False,
                     edge_color="dimgray",
                     node_color=node_color,
                     node_size=node_size)
    plt.axis('off')